In [2]:
import pandas as pd
import numpy as np
import joblib as jb

In [3]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test = test.filter(regex=r'feature', axis=1)

In [4]:
feature_groups = ['feature_intelligence', 'feature_charisma', 'feature_strength', 'feature_dexterity', 'feature_constitution', 'feature_wisdom']

In [5]:
from scipy.stats import spearmanr


In [6]:
X_train0.head()

feature_intelligence1  feature_intelligence2  feature_intelligence3  \
0                   0.00                   0.50                   0.25   
1                   0.00                   0.00                   0.00   
2                   0.25                   0.50                   0.25   
3                   1.00                   0.00                   0.00   
4                   0.25                   0.25                   0.25   

   feature_intelligence4  feature_intelligence5  feature_intelligence6  \
0                   0.00                    0.5                   0.25   
1                   0.25                    0.5                   0.00   
2                   0.25                    1.0                   0.75   
3                   0.50                    0.5                   0.25   
4                   0.25                    0.0                   0.25   

   feature_intelligence7  feature_intelligence8  feature_intelligence9  \
0                   0.25                   0.25                   0.75   
1                   0.00                   0.25                   0.50   
2                   0.75                   0.25                   0.00   
3                   0.25                   0.75                   0.25   
4                   0.50                   0.25                   0.25   

   feature_intelligence10  ...  feature_wisdom37  feature_wisdom38  \
0                    0.75  ...              1.00              1.00   
1                    0.50  ...              0.75              1.00   
2                    0.25  ...              0.50              0.25   
3                    0.50  ...              0.75              1.00   
4                    0.50  ...              0.50              0.75   

   feature_wisdom39  feature_wisdom40  feature_wisdom41  feature_wisdom42  \
0              1.00              0.75              0.50              0.75   
1              1.00              0.00              0.00              0.75   
2              0.50              0.00              0.00              0.50   
3              1.00              0.75              0.75              1.00   
4              0.75              0.25              0.50              0.75   

   feature_wisdom43  feature_wisdom44  feature_wisdom45  feature_wisdom46  
0              0.50              1.00              0.50              0.75  
1              0.25              0.00              0.25              1.00  
2              1.00              0.00              0.25              0.75  
3              1.00              0.75              1.00              1.00  
4              0.00              0.50              0.25              0.75  

[5 rows x 310 columns]

# LGBM para 1 grupo

In [9]:
from lightgbm import LGBMRegressor

for group in feature_groups:
    num_leaves, min_data_in_leaf, learning_rate = [130, 369, 0.009]
    
    
    mdl = LGBMRegressor(num_leaves=num_leaves,min_data_in_leaf=min_data_in_leaf, learning_rate=learning_rate, 
                        n_estimators=100, random_state=0)
    mdl.fit(X_train0.filter(regex=group,axis=1), y_train0)

    p = mdl.predict(X_train1.filter(regex=group,axis=1))
    model_name_train1 = "./preds_train1/lgbm_feature_groups_{}.pkl.z".format(group) 
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1.filter(regex=group,axis=1))
    model_name_val1 = "./preds_val1/lgbm_feature_groups_{}.pkl.z".format(group) 
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test.filter(regex=group,axis=1))
    model_name_test = "./preds_test/lgbm_feature_groups_{}.pkl.z".format(group) 
    jb.dump(p, model_name_test)

    print(group, metric)
    print()

   

feature_intelligence 0.0011307817762232634

feature_charisma 0.011496670691328858

feature_strength 0.008930369512651271

feature_dexterity 0.010307854965770625

feature_constitution 0.004058911449155239

feature_wisdom 0.009246470565812874



# LGBM com 2 grupos

In [19]:
from lightgbm import LGBMRegressor
from itertools import combinations

for group1, group2 in combinations(feature_groups, 2):
    num_leaves, min_data_in_leaf, learning_rate = [130, 369, 0.009]
    
    
    mdl = LGBMRegressor(num_leaves=num_leaves,min_data_in_leaf=min_data_in_leaf, learning_rate=learning_rate, 
                        n_estimators=100, random_state=0)
    
    selected = []
    for f in X_train0.columns:
        if group1 in f or group2 in f:
            selected.append(f)
    #print(selected)
    
    mdl.fit(X_train0[selected], y_train0)

    p = mdl.predict(X_train1[selected])
    model_name_train1 = "./preds_train1/lgbm_feature_groups_{}_{}.pkl.z".format(group1, group2) 
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1[selected])
    model_name_val1 = "./preds_val1/lgbm_feature_groups_{}_{}.pkl.z".format(group1, group2) 
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test[selected])
    model_name_test = "./preds_test/lgbm_feature_groups_{}_{}.pkl.z".format(group1, group2) 
    jb.dump(p, model_name_test)

    print(group1, group2, metric)
    print()

   

feature_intelligence feature_charisma 0.01570166591987752

feature_intelligence feature_strength 0.011905010595306915

feature_intelligence feature_dexterity 0.01292691192706217

feature_intelligence feature_constitution 0.01112005771078594

feature_intelligence feature_wisdom 0.01294216028109883

feature_charisma feature_strength 0.012214358174381589

feature_charisma feature_dexterity 0.023059135465429276

feature_charisma feature_constitution 0.01166893767076814

feature_charisma feature_wisdom 0.015957861557632725

feature_strength feature_dexterity 0.01783277382774583

feature_strength feature_constitution 0.007914137169913373

feature_strength feature_wisdom 0.013070252742486422

feature_dexterity feature_constitution 0.01844055715452453

feature_dexterity feature_wisdom 0.01978255235379457

feature_constitution feature_wisdom 0.012823284581624994



# LGBM com 3 grupos

In [21]:
from lightgbm import LGBMRegressor
from itertools import combinations

for group1, group2, group3 in combinations(feature_groups, 3):
    num_leaves, min_data_in_leaf, learning_rate = [130, 369, 0.009]
    
    
    mdl = LGBMRegressor(num_leaves=num_leaves,min_data_in_leaf=min_data_in_leaf, learning_rate=learning_rate, 
                        n_estimators=100, random_state=0)
    
    selected = []
    for f in X_train0.columns:
        if group1 in f or group2 in f or group3 in f:
            selected.append(f)
    #print(selected)
    
    mdl.fit(X_train0[selected], y_train0)

    p = mdl.predict(X_train1[selected])
    model_name_train1 = "./preds_train1/lgbm_feature_groups_{}_{}_{}.pkl.z".format(group1, group2, group3) 
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1[selected])
    model_name_val1 = "./preds_val1/lgbm_feature_groups_{}_{}_{}.pkl.z".format(group1, group2, group3) 
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test[selected])
    model_name_test = "./preds_test/lgbm_feature_groups_{}_{}_{}.pkl.z".format(group1, group2, group3) 
    jb.dump(p, model_name_test)

    print(group1, group2, group3, metric)
    print()

   

feature_intelligence feature_charisma feature_strength 0.017610286768396866

feature_intelligence feature_charisma feature_dexterity 0.022131277141476433

feature_intelligence feature_charisma feature_constitution 0.017351418006508183

feature_intelligence feature_charisma feature_wisdom 0.01592444757611113

feature_intelligence feature_strength feature_dexterity 0.018698960916808106

feature_intelligence feature_strength feature_constitution 0.013484357609494095

feature_intelligence feature_strength feature_wisdom 0.012699936764221853

feature_intelligence feature_dexterity feature_constitution 0.02059112975354937

feature_intelligence feature_dexterity feature_wisdom 0.020886540814831975

feature_intelligence feature_constitution feature_wisdom 0.013115500496018038

feature_charisma feature_strength feature_dexterity 0.02336463190302938

feature_charisma feature_strength feature_constitution 0.01250284217478524

feature_charisma feature_strength feature_wisdom 0.013695378108954704

